# README file
Changes: 
1. Topology file: Different position restraints
2. GPU-related options
3. Maximum number of iterations

In [156]:
# original vs modified
!diff --color original/README run1/README.sh

6c6
< # This folder contains GROMACS formatted force fields, a pre-optimized PDB structure, and GROMACS inputs.
---
> # This folder contains GROMACS formatted CHARMM36 force fields, a pre-optimized PDB structure, and GROMACS inputs.
26c26
< gmx_d mdrun -v -deffnm ${mini_prefix}
---
> gmx mdrun -v -deffnm ${mini_prefix} -nt 1
38,39c38,39
<     gmx grompp -f ${istep}.mdp -o ${istep}.tpr -c ${pstep}.gro -r ${rest_prefix}.gro -p topol.top -n index.ndx
<     gmx mdrun -v -deffnm ${istep}
---
>     gmx grompp -f ${istep}.mdp -o ${istep}.tpr -c ${pstep}.gro -r ${rest_prefix}.gro -p topol.top -n index.ndx -maxwarn 2
>     gmx mdrun -v -deffnm ${istep} -gpu_id 0 -ntomp 16 -bonded gpu -nb gpu -pme gpu -pmefft gpu -pin on -pinoffset 16 -pinstride 1
45c45
< set cntmax = 10
---
> set cntmax = 1
54c54
<         gmx grompp -f ${prod_prefix}.mdp -o ${istep}.tpr -c ${pstep}.gro -p topol.top -n index.ndx
---
>         gmx grompp -f ${prod_prefix}.mdp -o ${istep}.tpr -c ${pstep}.gro -r ${pstep}.gro -p to

# Step 6.4 Equilibration
1. Position restraint force constant *($kJ\ mol^{-1}\ nm^{-2}$)*
   - Backbone: 500 -> 2000
   - Side chain: 200 -> 1000
   - Lipid: 200 -> 0
   - (Dihedral restraint) Lipid: 200 -> 0
2. Run steps
   - nsteps: 250000 -> 10000000 (0.5ns -> 20ns)
   - nstxout-compressed: 5000 -> 50000 *(Compressed trajectory file every 0.1 ns (.xtc file))*
   - nstxtcout: 0 -> 5000 *(Frequency to write coordinates to xtc trajectory)*
   - nstenergy: 1000 -> 50000 *(Frequency to write energies to energy file)*
   - nstlog: 1000 -> 50000 *(Frequency to write energies to log file)*
4. Coupling *(Rescale particles velocity)*
    - tcoupl: v-rescale -> berendsen
    - vcoupl: C-rescale -> berendsen

In [47]:
# Original vs modified
!diff --color original/step6.4_equilibration.mdp run1/step6.4_equilibration.mdp 

1c1
< define                  = -DPOSRES -DPOSRES_FC_BB=500.0 -DPOSRES_FC_SC=200.0 -DPOSRES_FC_LIPID=200.0 -DDIHRES -DDIHRES_FC=200.0
---
> define                  = -DPOSRES -DPOSRES_FC_BB=2000.0 -DPOSRES_FC_SC=1000.0 -DPOSRES_FC_LIPID=0.0 -DDIHRES -DDIHRES_FC=0
4,6c4,7
< nsteps                  = 250000
< nstxout-compressed      = 5000
< nstxout                 = 0
---
> nsteps                  = 10000000	    ; 20ns
> 
> 
> ;nstxtcout               = 5000
9,11c10,14
< nstcalcenergy           = 100
< nstenergy               = 1000
< nstlog                  = 1000
---
> ;nstcalcenergy           = 100
> nstenergy               = 50000
> nstlog                  = 50000
> nstxout-compressed      = 50000             ; compressed trajectory file every 0.1 ns (.xtc file) 
> 
23c26
< tcoupl                  = v-rescale
---
> tcoupl                  = berendsen
28c31
< pcoupl                  = C-rescale
---
> pcoupl                  = berendsen


# Step 7 Production
1. Position restraint force constant:
   - Backbone: 0 -> 800
2. Run steps
   - nsteps: 250000 -> 10000000 (1ns -> 500ns)
   - nstcalcenergy: 100 -> 0 *(Frequency for calculating the energies)*
   - nstenergy: 1000 -> 50000 *(Frequency to write energies to energy file)*
   - nstlog: 1000 -> 50000 *(Frequency to write energies to log file)*
   - nstxout-compressed: 50000 -> 25000 *(Compressed trajectory file every 0.05 ns (.xtc file))*
4. Coupling
    - tcoupl: v-rescale -> Nose-Hoover
    - vcoupl: C-rescale -> Parrinello-Rahman
    - refcoord_scaling: no -> com *(Scale the center of mass of the reference coordinates with the scaling matrix of the pressure coupling)*
    - electric-field-z: 0 -> (0.10  0    0   0) *(field N/C, angular frequency, starting time, width of pulse)*

In [49]:
!diff --color original/step7_production.mdp run1/step7_production.mdp

0a1
> define                  = -DPOSRES -DPOSRES_FC_BB=800.0  -DPOSRES_FC_LIPID=0.0 -DDIHRES -DDIHRES_FC=0.0
3,4c4,5
< nsteps                  = 500000
< nstxout-compressed      = 50000
---
> nsteps                  = 250000000	    ; 500ns
> 
8,10c9,13
< nstcalcenergy           = 100
< nstenergy               = 1000
< nstlog                  = 1000
---
> ;nstcalcenergy           = 100
> nstenergy               = 50000
> nstlog                  = 50000
> nstxout-compressed      = 25000             ; compressed trajectory file every 0.05 ns (.xtc file) 
> 
22c25
< tcoupl                  = v-rescale
---
> tcoupl                  = Nose-Hoover
27c30
< pcoupl                  = C-rescale
---
> pcoupl                  = Parrinello-Rahman
40a44,46
> refcoord_scaling        = com
> 
> electric-field-z        = 0.10  0    0   0 


# Position restraints 
- Only impose restraints on residue 649, 651, 659, 660

In [145]:
# original 
!wc -l original/toppar/PROA.itp
!grep -A 10 -n "position_restraints" original/toppar/PROA.itp

    6606 original/toppar/PROA.itp
6279:[ position_restraints ]
6280-    1     1    POSRES_FC_BB    POSRES_FC_BB    POSRES_FC_BB   
6281-    5     1    POSRES_FC_BB    POSRES_FC_BB    POSRES_FC_BB   
6282-    7     1    POSRES_FC_SC    POSRES_FC_SC    POSRES_FC_SC   
6283-   11     1    POSRES_FC_BB    POSRES_FC_BB    POSRES_FC_BB   
6284-   12     1    POSRES_FC_BB    POSRES_FC_BB    POSRES_FC_BB   
6285-   13     1    POSRES_FC_BB    POSRES_FC_BB    POSRES_FC_BB   
6286-   15     1    POSRES_FC_BB    POSRES_FC_BB    POSRES_FC_BB   
6287-   17     1    POSRES_FC_SC    POSRES_FC_SC    POSRES_FC_SC   
6288-   20     1    POSRES_FC_SC    POSRES_FC_SC    POSRES_FC_SC   
6289-   21     1    POSRES_FC_SC    POSRES_FC_SC    POSRES_FC_SC   


In [147]:
# modified
!wc run1/toppar_res46/PROA.itp
!grep -A 10 -n "position_restraints" run1/toppar_res46/PROA.itp

    6283   29299  190359 run1/toppar_res46/PROA.itp
6279:[ position_restraints ]     
6280-  649     1    POSRES_FC_BB    POSRES_FC_BB    POSRES_FC_BB   
6281-  651     1    POSRES_FC_BB    POSRES_FC_BB    POSRES_FC_BB   
6282-  659     1    POSRES_FC_BB    POSRES_FC_BB    POSRES_FC_BB   
6283-  660     1    POSRES_FC_BB    POSRES_FC_BB    POSRES_FC_BB   
6284-#endif
